In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D
import h5py
import scipy.optimize as optimize
from IPython.display import clear_output
import time
import os

In [ ]:
#os.listdir("./angle_sweep/field_sweep/")
os.listdir("./field_sweep/")

In [ ]:
# Load data
def load_data(name):

    try:
        with h5py.File(f'{name}', 'r') as f:
            applied_field = f['applied_field'][:]
            waveforms = f['waveforms'][:]
            frequency = f['frequency'][:]
            #applied_field = f['applied_field'][:]
        print("Data loaded!")
        print("Time:", time.strftime("%H:%M:%S", time.localtime()))
        return applied_field, frequency, waveforms
    except Exception as e:
        print("File not found!", e)


In [ ]:
# 2D spectrum plot

'''
x_values = applied_field[:41]#angle+77
y_values = frequency/1e9
X, Y = np.meshgrid(x_values, y_values)
'''
name = './field_sweep/3-5_X6_L2R3_2_0mA_82deg.h5'

x_values, y_values, waveforms = load_data(name)
X, Y = np.meshgrid(x_values, y_values*1e-9)

plt.pcolormesh(X, Y, waveforms.T, shading='nearest', vmax=-70, vmin=-78)
plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Applied field [T]')
plt.xlim(0.35, 0.68)
plt.ylabel('Frequency [GHz]')
plt.tight_layout()
plt.savefig(f"./plots/{name.split('/')[-1]}.png", dpi=400)
plt.show()

In [ ]:
# 2D spectrum plot

'''
x_values = applied_field[:41]#angle+77
y_values = frequency/1e9
X, Y = np.meshgrid(x_values, y_values)
'''
name = './bias_sweep/3-10_X6_L2R4_76deg_field_3_5A.h5'


with h5py.File(f'{name}', 'r') as f:
            applied_field = f['applied_field'][:]
            waveforms = f['waveforms'][:]
            frequency = f['frequency'][:]
            bias_current = f['bias_range'][:]
            
X, Y = np.meshgrid(bias_current*1000, frequency*1e-9)

plt.pcolormesh(X, Y, waveforms.T, shading='gouraud',vmax=-70, vmin=-76, cmap='jet')
plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Bias current [mA]')
plt.ylabel('Frequency [GHz]')
plt.ylim(6.6,7.1)
plt.tight_layout()
#plt.savefig(f"./plots/{name.split('/')[-1]}.png", dpi=400)
plt.show()

In [ ]:
# 2D spectrum plot
import wave
from numpy import angle


name = './angle_sweep/3-11_X6_L2R4_2_0mA_3_5A_68_deg.h5'

with h5py.File(f'{name}', 'r') as f:
            applied_field = f['applied_field'][:]
            waveforms = f['waveforms'][:]
            frequency = f['frequency'][:]
angle_range = np.linspace(68, 86, int((18/0.05)))
            
X, Y = np.meshgrid(angle_range, frequency*1e-9)

running_avg = np.zeros_like(waveforms)

fs = 2
for i in range(len(frequency-fs)):
    for j in range(len(applied_field-fs)):
        running_avg[j,i] = np.mean(waveforms[j:j+fs,i:i+fs])


plt.pcolormesh(X, Y, running_avg.T, shading='nearest', vmax=-70, vmin=-80, cmap='viridis')
plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Angle Range [mA]')
plt.ylabel('Frequency [GHz]')
plt.tight_layout()
plt.savefig(f"./plots/{name.split('/')[-1]}.png", dpi=600)
plt.show()

In [ ]:
plt.plot(frequency*1e-6, waveforms[95])
running_avg = np.zeros_like(waveforms[95])
for i in range(len(waveforms[95])-5):
    running_avg[i] = np.mean(waveforms[95][i:i+5])
running_avg[np.where(running_avg==0)] = -70
plt.plot(frequency*1e-6, running_avg)
plt.xlim(8190,8225)
plt.hlines(np.max(running_avg), 8190, 8225, color='tab:red', ls='--')
plt.hlines(np.max(running_avg)-3, 8190, 8225, color='tab:cyan', ls='--')
half_power = (np.where(np.isclose(running_avg, np.max(running_avg)-3, atol=0.1)==True)[0])
plt.vlines(frequency[half_power[0]]*1e-6, -79, -68, color='tab:gray', ls='--')
plt.vlines(frequency[half_power[1]]*1e-6, -79, -68, color='tab:gray', ls='--')
print("Line width:", (frequency[half_power[1]]-frequency[half_power[0]])*1e-6)
plt.ylim(-79, -68)
plt.show()

In [ ]:
np.isclose(running_avg, np.max(running_avg)-3, atol=0.02)

In [ ]:
np.max(running_avg)

In [ ]:
from IPython.display import clear_output
from scipy import signal
'''
Extract specgtrum at  given applied field across all the files.

The data is plotted as out of plane angle vs frequency.
'''

# filtering
filtering = False
b, a = signal.butter(4, [2.35e9, 2.4e9], 'bandstop', analog=True)
w, h = signal.freqs(b, a, worN=np.logspace(np.log10(2e9), np.log10(6e9), 4000))

applied_fields_select = 0.5#[T]                      # title states the apprix field strength


selected_waveforms = np.zeros((len(os.listdir("./angle_sweep/field_sweep/")), 4000))

for i, name in enumerate(os.listdir("./angle_sweep/field_sweep/")):
    applied_field, frequency, waveforms = load_data('./angle_sweep/field_sweep/' + name)
    
    
    #np.isclose(applied_field, applied_fields_select, atol=0.001)
    ind = np.where(np.isclose(applied_field, applied_fields_select, atol=0.01))[0][0]
    print(ind)
    
    if filtering:
        selected_waveforms[i] =  waveforms[ind]+ 20 * np.log10(abs(h))
    else:
        selected_waveforms[i] = waveforms[ind] 
    clear_output(wait=True)



X, Y = np.meshgrid(np.arange(0,len(os.listdir("./angle_sweep/field_sweep/")))*0.1+74, frequency*1e-9) # note the start angle is 78 with steps of 0.1

plt.title(f"Applied field @ {applied_fields_select} T")

plt.pcolormesh(X, Y, selected_waveforms.T, shading='gouraud', vmin=-75, vmax=-60)

plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Out-of-Plane angle [$ \degree$]')
plt.ylabel('Frequency [GHz]')
plt.tight_layout()
plt.savefig(f"./plots/angle_sweep_{applied_fields_select}T.png", dpi=300)
plt.show()

In [ ]:
# Plotting in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, ft in enumerate(waveforms):
    ax.plot(frequency/1e9, applied_field[i]*1000, zs=ft, zorder=-i,lw=0.5) #label=f"Signal {i+1}"
    
ax.set_xlabel('Frequency [GHz]')
ax.set_zlabel('Amplitude [dBm]')
ax.set_ylabel('Applied field[mT]')
ax.set_title('3D Plot of Captured Spectrum')
plt.tight_layout()

plt.show()

In [ ]:
colormap = 'Viridis'
vmin, vmax = -80, -40  # Fixed range in dBm


X, Y = np.meshgrid(frequency / 1e9, applied_field)
Z = np.array(waveforms)

fig = go.Figure(data=[go.Surface(
    x=X, y=Y, z=Z,
    colorscale=colormap,
    cmin=vmin, cmax=vmax,
    colorbar=dict(title="Amplitude [dBm]")
)])

fig.update_layout(
    width=800,
    height=600,
    scene=dict(
        xaxis_title='Frequency [GHz]',
        yaxis_title='Applied field [mT]',
        zaxis_title='Amplitude [dBm]'
    ),
    title='Interactive 3D Plot of Captured Spectrum',
    autosize=True, 
    showlegend=False
)

# Show the plot
fig.show()


## Background noise removal test for standard sweeps

In [ ]:
# Load the data
names = os.listdir("./angle_sweep/field_sweep/X6_L2R4")

angle_range = np.linspace(float(names[0].split("_")[-1].split("deg")[0]), float(names[-1].split("_")[-1].split("deg")[0]), len(names))

names = os.listdir("./angle_sweep/field_sweep/X6_L2R4")
with h5py.File(f'./angle_sweep/field_sweep/X6_L2R4/{names[0]}', 'r') as f:
    applied_field = f['applied_field'][:]
    waveforms = f['waveforms'][:]
    frequency = f['frequency'][:]

app_field_avg = np.zeros_like(applied_field)
wfm_avg = np.zeros_like(waveforms)
freq_avg = np.zeros_like(frequency)

for name in names:
    with h5py.File(f'./angle_sweep/field_sweep/X6_L2R4/{name}', 'r') as f:
        applied_field = f['applied_field'][:]
        waveforms = f['waveforms'][:]
        frequency = f['frequency'][:]
    
    app_field_avg = np.add(app_field_avg, applied_field)
    wfm_avg = np.add(wfm_avg, waveforms)
    freq_avg = np.add(freq_avg, frequency)
    
    
app_field_avg = app_field_avg/len(names)
wfm_avg = wfm_avg/len(names)
freq_avg = freq_avg/len(names)


X, Y = np.meshgrid(app_field_avg*1000, freq_avg*1e-9)
plt.pcolormesh(X, Y, wfm_avg.T, shading='nearest', cmap='viridis')
plt.colorbar(label='Amplitude [dBm]')
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.tight_layout()
#plt.savefig(f"./plots/angle_sweep_avg.png", dpi=600)
plt.show()


In [ ]:
wfm[8009:9000] = 0


In [ ]:
a = np.array([1,2,3])
np.where([1,2,3] == 1, a, a+1)

In [ ]:
import os
e = 7
path = f'./angle_sweep/field_sweep/X6_L6R6'
names = os.listdir(path)
with h5py.File(path + f"/{names[e]}", 'r') as f:
    applied_field = f['applied_field'][:]
    waveforms = f['waveforms'][:]
    frequency = f['frequency'][:]

print(names[e])

wfm_avg = get_wfm_background(path)

fig, axs = plt.subplots(2,1, figsize=(6, 9))

# First subplot, average
X, Y = np.meshgrid(applied_field*1000, frequency*1e-9)
c1 = axs[0].pcolormesh(X, Y, waveforms.T - wfm_avg.T, shading='nearest', cmap='viridis', vmin=0, vmax=7)
fig.colorbar(c1, ax=axs[0])
axs[0].set_xlabel('Applied field [mT]')
axs[0].set_ylabel('Frequency [GHz]')
axs[0].set_title('Average noise floor subtracted')

# Second subplot, raw
X, Y = np.meshgrid(applied_field*1000, frequency*1e-9)
c2 = axs[1].pcolormesh(X, Y, waveforms.T, shading='auto', cmap='viridis')
fig.colorbar(c2, ax=axs[1])
axs[1].set_xlabel('Applied field [mT]')
axs[1].set_ylabel('Frequency [GHz]')
axs[1].set_title('Raw data')


plt.tight_layout()
#plt.savefig("noise_removale_examples.png", dpi=600)
plt.show()

In [ ]:
threshold = 3

sub_waveforms = waveforms - wfm_avg
field_data = applied_field[np.where(sub_waveforms > threshold)[0][1:]]*1e3
freq_data = frequency[np.where(sub_waveforms > threshold)[1][1:]]*1e-9

a, b = optimize.curve_fit(lambda x, a, b: a*x+b, field_data, freq_data)[0]

plt.plot(applied_field*1e3, a*applied_field*1e3+b)
plt.annotate(f"Linear fit:\ny = {a:.4e} x + {b:.4e}", xy=(0.1, 0.8), xycoords='axes fraction')

plt.scatter(field_data, freq_data, color="tab:orange", zorder=10)
plt.ylabel('Frequency [GHz]')
plt.xlabel('Applied field [mT]')
plt.show()

In [ ]:
from re import sub
import numpy as np
from scipy.optimize import curve_fit
def linear_model(x, m, b):
    return m * x + b


def get_wfm_background(path):

    names = os.listdir(path)

    with h5py.File(f'{path}/{names[0]}', 'r') as f:
        #applied_field = f['applied_field'][:]
        waveforms = f['waveforms'][:]
        #frequency = f['frequency'][:]

    #app_field_avg = np.zeros_like(applied_field)
    wfm_avg = np.zeros_like(waveforms)
    #freq_avg = np.zeros_like(frequency)

    for name in names:
        with h5py.File(f'{path}/{name}', 'r') as f:
            #applied_field = f['applied_field'][:]
            waveforms = f['waveforms'][:]
            #frequency = f['frequency'][:]
        
        #app_field_avg = np.add(app_field_avg, applied_field)
        wfm_avg = np.add(wfm_avg, waveforms)
        #freq_avg = np.add(freq_avg, frequency)
        
        
    #app_field_avg = app_field_avg/len(names)
    wfm_avg = wfm_avg/len(names)
    #freq_avg = freq_avg/len(names)

    return wfm_avg

def find_linje(path, sample_name, threshold = 4, plots = True):
    
    
    names = os.listdir(path)
    wfm_avg = get_wfm_background(path)
    
    
    with h5py.File(f'{path}/{sample_name}', 'r') as f:
        applied_field = f['applied_field'][:]
        waveforms = f['waveforms'][:]
        frequency = f['frequency'][:]

    #waveforms[:,8000:9000]= 0
    
    sub_waveforms = waveforms - wfm_avg
    
    sub_waveforms[:,6500:6800]= 0   #
    sub_waveforms[:, 13500:] = 0    # 8 GHz LPF
    
    sub_waveforms[32:] = 0

    field_data = applied_field[np.where((sub_waveforms < 6) & (sub_waveforms > threshold))[0]]*1e3
    freq_data = frequency[np.where((sub_waveforms < 6) & (sub_waveforms > threshold))[1]]*1e-9

    # Første iteration
    initial_residuals = np.abs(freq_data - linear_model(field_data, 0.01, 1))  # første gæt...
    params, covariance = curve_fit(linear_model, field_data, freq_data, sigma=initial_residuals, absolute_sigma=True)
    m_fit1, b_fit1 = params

    # Anden iteration
    step_residuals = np.abs(freq_data - linear_model(field_data, m_fit1, b_fit1))
    params, covariance = curve_fit(linear_model, field_data, freq_data, sigma=-step_residuals)
    m_fit2, b_fit2 = params

    # Tredje iteration
    step_residuals = np.abs(freq_data - linear_model(field_data, m_fit2, b_fit2))
    params, covariance = curve_fit(linear_model, field_data, freq_data, sigma=-step_residuals)
    m_fit3, b_fit3 = params

    close_datapoints = np.where(step_residuals < 0.07)
    
    # Plot
    if plots:
        plt.plot(applied_field*1e3, m_fit1*applied_field*1e3+b_fit1, label="Step 1")
        plt.plot(applied_field*1e3, m_fit2*applied_field*1e3+b_fit2, label="Step 2")
        plt.plot(applied_field*1e3, m_fit3*applied_field*1e3+b_fit3, label="Step 3")
        plt.annotate(f"Linear fit:\ny = {m_fit3:.4e} x + {b_fit3:.4e}", xy=(0.1, 0.8), xycoords='axes fraction')
        plt.legend()

        plt.scatter(field_data, freq_data, color="tab:orange", zorder=10)
        plt.ylabel('Frequency [GHz]')
        plt.xlabel('Applied field [mT]')
        plt.ylim(3,9)
        plt.xlim(applied_field[0]*1e3, applied_field[-1]*1e3)
        
        plt.show()
    return m_fit3, b_fit3, field_data[close_datapoints], freq_data[close_datapoints]


In [ ]:
path = './angle_sweep/field_sweep/X6_L2R2'
names = os.listdir(path)
for name in names:
    print(name)
    find_linje(path, name, 3)




## L2R4

In [ ]:
'''
Function paramters to get the plot for L2R4

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L2R4'
names = os.listdir(path)
threshold = 3
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 3, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = "OOP: " + str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show()   

## L2R1

In [ ]:
'''
Function paramters to get the plot for L2R1

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3.1 in find_linje
'''



path = './angle_sweep/field_sweep/X6_L2R1'
names = os.listdir(path)
threshold = 3
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 3.1, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show()   

## L2R3

In [ ]:
'''
Function paramters to get the plot for L2R3

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 2 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L2R3'
names = os.listdir(path)

for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 2, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show()   

## L2R5

In [ ]:
'''
Function paramters to get the plot for L2R5

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L2R5'
names = os.listdir(path)
threshold = 3
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 3, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show()   

## L2R6

In [ ]:
'''
Function paramters to get the plot for L2R6

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L2R6'
names = os.listdir(path)
threshold = 3
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 3, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = "OOP: " + str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show()   

## L2R7

In [ ]:
'''
Function paramters to get the plot for L2R6

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.15)
    
    # note threshold = 3.6 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L2R7'
names = os.listdir(path)
threshold = 3
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show()   

## R2s @ angle

In [ ]:
path = './angle_sweep/field_sweep/'
names = os.listdir(path)

angle = 76.0 # be exact, this is name of the file

file_names = []


ases = []
bses = []
tses = []
for name in names:
    if (name).startswith("X6_L2"):
        print(name)
        #print(os.listdir(path+name))
        for file in os.listdir(path+name):
            if file.endswith(str(angle)+"deg.h5"):
                #(name).startswith("X6_L2"):
                if name == "X6_L2R3" or name == "X6_L2R3_open" or name == "X6_L2R2":
                    break
                print(file)
                
                
                threshold = 3
                a, b, data_x, data_y = find_linje(path+"/"+name, file, threshold, plots = False)
                plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = name.replace("X6_L2",""))
                plt.scatter(data_x, data_y, zorder=10)
                
                ases.append(a)
                bses.append(b)
                
                tses.append(a*500+b)
                

#print(np.mean(ases), np.std(ases), np.mean(bses), np.std(bses))
print(np.std(tses))
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "Sample #")
plt.grid()
plt.tight_layout()
plt.savefig("./angle_sweep/field_sweep/plots/L2RX_sample_sweep_80deg.png", dpi=600)
plt.show()

In [ ]:
from cProfile import label


path = './angle_sweep/field_sweep/'
names = os.listdir(path)

angle = 76.0 # be exact, this is name of the file

file_names = []

plt.figure(figsize=(7,4))


fake_x = np.linspace(applied_field[0],applied_field[-1],100)*1e3
avg_wfm = np.zeros_like(fake_x)

ases = []
bses = []
tses = []
for name in names[:-1]:
    if (name).startswith("X6_L2"):
        print(name)
        #print(os.listdir(path+name))
        for file in os.listdir(path+name):
            if file.endswith(str(angle)+"deg.h5"):
                #(name).startswith("X6_L2"):
                if name == "X6_L2R3" or name == "X6_L2R3_open" or name == "X6_L2R2":
                    break
                print(file)
                
                
                threshold = 3
                a, b, data_x, data_y = find_linje(path+"/"+name, file, threshold, plots = False)
                #plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = name.replace("X6_L2",""))
                #plt.scatter(data_x, data_y, zorder=10)
                
                #plt.plot(fake_x, a*fake_x+b)
                #avg_wfm += a*fake_x+b
                ases.append(a)
                bses.append(b)
                
                tses.append(a*500+b)
                

for a,b in zip(ases, bses):
    avg_wfm += a*fake_x+b
avg_wfm /= len(ases)


errors = []
for fx in fake_x:
    std_temp_var = []
    for a,b in zip(ases, bses):
        std_temp_var.append(a*fx+b)
    errors.append(3*np.std(std_temp_var))



plt.plot(fake_x,avg_wfm, label = "Average frequency")
plt.fill_between(fake_x, avg_wfm-errors,avg_wfm+errors,alpha = 0.2, label = "3$\sigma$")

#print(np.mean(ases), np.std(ases), np.mean(bses), np.std(bses))
#print(np.std(tses))
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(fancybox = True)
plt.grid()
plt.tight_layout()
plt.savefig("./angle_sweep/field_sweep/plots/L2_frequency_STD_76deg.png", dpi=600)
plt.show()

In [ ]:
np.std(std_temp_var)

In [ ]:
np.mean(ases)*(0.002*0.1525*1e3)

In [ ]:
f"{(3e9/2000):.4e}"

## L1R1

In [ ]:
'''
Function paramters to get the plot for L2R5

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L1R1'
names = os.listdir(path)
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 2, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show() 

## L1R2

In [ ]:
'''
Function paramters to get the plot for L2R5

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L1R2'
names = os.listdir(path)
for i, name in enumerate(names[:-2]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 2, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.tight_layout()
plt.show() 

## L1R3

In [ ]:
'''
Function paramters to get the plot for L1R3

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L1R3'
names = os.listdir(path)
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 2, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show() 

## L1R4

In [ ]:
'''
Function paramters to get the plot for L1R3

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L1R4'
names = os.listdir(path)
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 2, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show() 

## L1R5

In [ ]:
'''
Function paramters to get the plot for L1R3

    sub_waveforms[:,6500:6800]= 0   # set approx 5,4GHz to 0
    sub_waveforms[32:] = 0          # set lower freq to 0
    
    
    close_datapoints = np.where(step_residuals < 0.07)
    
    note threshold = 3 in find_linje
'''

path = './angle_sweep/field_sweep/X6_L1R5'
names = os.listdir(path)
for i, name in enumerate(names[0:-3]):
    #print(name)
    a, b, data_x, data_y = find_linje(path, name, threshold = 2, plots = False)
    
    plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = str(angle_range[i]) + "$\degree$")
    plt.scatter(data_x, data_y, zorder=10)
    
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "OOP")
plt.grid()
plt.show() 

## L1s @ angle

In [ ]:
path = './angle_sweep/field_sweep/'
names = os.listdir(path)

angle = 76.5 # be exact, this is name of the file

file_names = []


ases = []
bses = []
tses = []

# this is used to get the specific angle from the folders of different sampels
for name in names:
    if (name).startswith("X6_L1"):
        print(name)
        #print(os.listdir(path+name))
        for file in os.listdir(path+name):
            if file.endswith(str(angle)+"deg.h5"):
                #(name).startswith("X6_L2"):
                if name == "X6_L2R3":
                    break
                print(file)
                
                
                
                a, b, data_x, data_y = find_linje(path+"/"+name, file, threshold, plots = False)
                plt.plot(applied_field[np.where(sub_waveforms > threshold)[0]]*1e3, a*applied_field[np.where(sub_waveforms > threshold)[0]]*1e3+b, label = name)
                plt.scatter(data_x, data_y, zorder=10)
                print(len(data_x))
                # ases.append(a)
                bses.append(b)
                
                tses.append(a*500+b)
                

#print(np.mean(ases), np.std(ases), np.mean(bses), np.std(bses))
print(np.std(tses))
plt.xlabel('Applied field [mT]')
plt.ylabel('Frequency [GHz]')
plt.legend(bbox_to_anchor=(1, 1.02), fancybox = True, title = "Sample #")
plt.tight_layout()
plt.savefig("./angle_sweep/field_sweep/plots/L1RX_sample_sweep_76_5deg.png", dpi=600)
plt.show()